## Analytics of Game results

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import itertools
import json
import re
import sys
sys.path.append('../')

from utils.analytics import *



## Plot mistakes

## Analyze Single Circle for dominated Strategies

In [ ]:
'''
- Mistake 1: Making an offer worse than your previous offer. This occurs when you reject an offer better for you than the one you subsequently propose. 
- Mistake 2: Making an offer worse for you than your outside offer. This happens if you propose giving away so much that what you keep is worth less than your guaranteed alternative, which is your outside offer.
- Mistake 3: Offering no items or all items. Offering nothing (or everything) to the opponent (in the early or middle rounds) can be a clear suboptimal move. 
- Mistake 4: Accepting an offer worse for you than your outside offer. This occurs if you accept a division that yields a payoff lower than your guaranteed fallback.
- Mistake 5: Walking away from an offer better than your outside offer. This occurs when you reject a division that actually yields a higher payoff than your fallback.
'''
circles = [0, 1, 3, 4, 5, 6]

for circle in circles:
    print(f"Analyzing circle {circle}")
    print("="*100)
    
    # Analyze circle with 4o
    json_FILE_PATH_4o = f"/Users/gabesmithline/Desktop/caif_negotiation/experiments/4o_final/4o_1_28_2025_100_circle_{circle}.json"
    mistake_counts_4o = analyze_circle6(json_FILE_PATH_4o)
    print(f"4o mistake counts: {mistake_counts_4o}")
    
    # Analyze circle with gemini_2.0
    json_FILE_PATH_gemini = f"/Users/gabesmithline/Desktop/caif_negotiation/experiments/gemini_2.0_final/gemini_2.0_1_28_2025_circle_{circle}.json"
    mistake_counts_gemini = analyze_circle6(json_FILE_PATH_gemini)
    print(f"gemini_2.0 mistake counts: {mistake_counts_gemini}")

    json_FILE_PATH_o3_mini = f"/Users/gabesmithline/Desktop/caif_negotiation/experiments/o3-mini/all_game_data_2_4_2025_100_o3-mini_circle_{circle}.json"
    mistake_counts_o3 = analyze_circle6(json_FILE_PATH_o3_mini)
    print(f"o3 mistake counts: {mistake_counts_o3}")


    
    # Prepare data for radar chart
    ds_data_for_circle = {
        f"4o_circle_{circle}": mistake_counts_4o,
        f"gemini_2.0_circle_{circle}": mistake_counts_gemini,
        f"o3-mini{circle}": mistake_counts_o3
    }
    
    # Plot a single radar chart comparing 4o and gemini_2.0 for this circle
    plot_mistakes_radar_multiple_circles(ds_data_for_circle, f"Circle {circle}")

## Plot Dominated Strats across multiple circles for different LLMs

In [ ]:
circles = [0, 1, 2, 3, 4, 5, 6]
stats_total = []  # will hold tuples of (stats, num_games)

for circle in circles:
    print(f"{'='*50} Circle: {circle} {'='*50}")
    #json_FILE_PATH = f"/Users/gabesmithline/Desktop/caif_negotiation/experiments/4o_final/4o_1_28_2025_100_circle_{circle}.json"
    #json_FILE_PATH = f"/Users/gabesmithline/Desktop/caif_negotiation/experiments/gemini_2.0_final/gemini_2.0_1_28_2025_circle_{circle}.json"
    json_FILE_PATH = f"/Users/gabesmithline/Desktop/caif_negotiation/experiments/o3-mini/all_game_data_2_4_2025_100_o3-mini_circle_{circle}.json"

    stats, num_games = count_game_actions(json_FILE_PATH)
    stats_total.append((stats, num_games))

    # Print table for that circle
    print_game_action_stats(stats)

# Now aggregate across all circles
agg_stats, total_games = aggregate_stats_across_circles(stats_total)
print("\n\nAggregated action counts across all circles:\n")
print_game_action_stats(agg_stats)

print("\n\nAverage actions per game across all circles:\n")
print_averages_across_all_games(agg_stats, total_games)

## Count game actions (ACCEPT, WALK, etc.) for a single circle

In [ ]:
compute_and_plot_metrics(
    #json_file_path = "/Users/gabesmithline/Desktop/caif_negotiation/experiments/gemini_2.0_final/gemini_2.0_1_28_2025_circle_6.json",
    json_file_path = "/Users/gabesmithline/Desktop/caif_negotiation/experiments/4o_final/4o_1_28_2025_100_circle_6.json",
    circle_name = "circle_6",
    llm_name = "4o",
    gamma = 0.9
)

##  Aggregate stats over multiple circles

In [ ]:
analyze_llm_all_circles(
        llm_name = "gemini_2.0_1_28_2025",
        base_path="/Users/gabesmithline/Desktop/caif_negotiation/experiments/gemini_2.0_final",
        circles=range(7)
    )


## Compute and Plot Detailed Metrics

In [ ]:
compare_llm_across_circles_with_stats_confint_and_distributions(
    #base_path="/Users/gabesmithline/Desktop/caif_negotiation/experiments/gemini_2.0_final/gemini_2.0_1_28_2025_circle_{circle}.json",
    #base_path= "/Users/gabesmithline/Desktop/caif_negotiation/experiments/o3-mini/all_game_data_2_4_2025_100_o3-mini_circle_{circle}.json",
    #base_path="/Users/gabesmithline/Desktop/caif_negotiation/experiments/4o_final/4o_1_28_2025_100_circle_{circle}.json",
    #base_path="/Users/gabesmithline/Desktop/caif_negotiation/experiments/4o_small_final/4o_2_6_2025_100_circle_{circle}_small.json",
    base_path="/Users/gabesmithline/Desktop/caif_negotiation/experiments/all_game_data_2_17_2025_300_4o_vs_4o_normal_distribution_circle_{circle}.json",

    circle_list=[0,1,2,3,4,5,6],
    llm_name="4o",
    gamma=0.9
)


In [ ]:
compare_llm_across_circles_with_stats_confint_and_distributions(
    #base_path="/Users/gabesmithline/Desktop/caif_negotiation/experiments/gemini_2.0_final/gemini_2.0_1_28_2025_circle_{circle}.json",
    #base_path= "/Users/gabesmithline/Desktop/caif_negotiation/experiments/o3-mini/all_game_data_2_4_2025_100_o3-mini_circle_{circle}.json",
    base_path="/Users/gabesmithline/Desktop/caif_negotiation/experiments/4o_final/4o_1_28_2025_100_circle_{circle}.json",
    #base_path="/Users/gabesmithline/Desktop/caif_negotiation/experiments/4o_small_final/4o_2_6_2025_100_circle_{circle}_small.json",
    #base_path="/Users/gabesmithline/Desktop/caif_negotiation/experiments/all_game_data_2_17_2025_300_4o_vs_4o_normal_distribution_circle_{circle}.json",

    circle_list=[0,1,2,3,4,5,6],
    llm_name="4o",
    gamma=0.9
)


## Further Analysis across all circles

In [ ]:

compare_llms_for_circle(
    base_path1 = "/Users/gabesmithline/Desktop/caif_negotiation/experiments/gemini_2.0_final/{llm}_1_28_2025_circle_{circle}.json",
    base_path2 = "/Users/gabesmithline/Desktop/caif_negotiation/experiments/4o_final/{llm}_1_28_2025_100_circle_{circle}.json",
    circle_number = 2,
    llm_list = ["gemini_2.0", "4o"],
    gamma = 0.9
)

## Detailed Metrics